In [1]:
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

In [2]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, output_size=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
        out = self.fc(hn[-1])
        return out

In [3]:
def make_multiple_predictions(input_sequence, num_predictions, city):

    city = city.upper()

    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float() 

    model = LSTMModel()
    model.load_state_dict(torch.load(f'./weights/city{city}.pth'))
    model.eval()

    seq_length = torch.tensor([len(input_sequence)])  

    extended_sequence = input_sequence.copy()

    with torch.no_grad():
        for _ in range(num_predictions):
            predicted_next = model(input_tensor, seq_length)
            predicted_next = predicted_next.squeeze().cpu().numpy()  
            predicted_next = [round(coordinate) for coordinate in predicted_next]  

            extended_sequence.append(predicted_next)

            input_tensor = torch.tensor(extended_sequence[-len(input_sequence):]).unsqueeze(0).float()
            seq_length = torch.tensor([len(input_tensor[0])])  # Update sequence length

    return extended_sequence


In [4]:
def make_prediction_city_A(input_sequence):
    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)]) 

    model = LSTMModel()

    model.load_state_dict(torch.load('./weights/cityA.pth'))
    model.eval()

    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)])  

    with torch.no_grad():
        predicted_next = model(input_tensor, seq_length)
        predicted_next = predicted_next.squeeze().cpu().numpy()  
        predicted_next = predicted_next.tolist()

    predicted_next = [round(coordinate) for coordinate in predicted_next]

    return predicted_next

In [5]:
def make_prediction_city_B(input_sequence):
    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)]) 

    model = LSTMModel()

    model.load_state_dict(torch.load('./weights/cityB.pth'))
    model.eval()

    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)])  

    with torch.no_grad():
        predicted_next = model(input_tensor, seq_length)
        predicted_next = predicted_next.squeeze().cpu().numpy()  
        predicted_next = predicted_next.tolist()

    predicted_next = [round(coordinate) for coordinate in predicted_next]

    return predicted_next

In [6]:
def make_prediction_city_C(input_sequence):
    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)]) 

    model = LSTMModel()

    model.load_state_dict(torch.load('./weights/cityC.pth'))
    model.eval()

    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)])  

    with torch.no_grad():
        predicted_next = model(input_tensor, seq_length)
        predicted_next = predicted_next.squeeze().cpu().numpy()  
        predicted_next = predicted_next.tolist()
    
    predicted_next = [round(coordinate) for coordinate in predicted_next]
        
    return predicted_next

In [7]:
def make_prediction_city_D(input_sequence):
    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)]) 

    model = LSTMModel()

    model.load_state_dict(torch.load('./weights/cityD.pth'))
    model.eval()

    input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()  
    seq_length = torch.tensor([len(input_sequence)])  

    with torch.no_grad():
        predicted_next = model(input_tensor, seq_length)
        predicted_next = predicted_next.squeeze().cpu().numpy()  
        predicted_next = predicted_next.tolist()
    
    predicted_next = [round(coordinate) for coordinate in predicted_next]

    return predicted_next

In [8]:
poi_dataA = pd.read_csv('../../Datasets/Task 1/POIdata_cityA.csv')
poi_dataB = pd.read_csv('../../Datasets/Task 1/POIdata_cityB.csv')
poi_dataC = pd.read_csv('../../Datasets/Task 1/POIdata_cityC.csv')
poi_dataD = pd.read_csv('../../Datasets/Task 1/POIdata_cityD.csv')
categories = pd.read_csv('../../Datasets/Task 1/POI_datacategories.csv', header=None)
categories = list(categories[0])
len(categories)

85

In [9]:
poi_listA = list(poi_dataA['category'])
poi_list_strA = []

for i in range(len(poi_listA)):
    index = poi_listA[i]-1
    name = categories[index]
    poi_list_strA.append(name)

poi_list_strA    

poi_dataA.drop(columns=['category'], inplace=True)
poi_dataA['category'] = poi_list_strA

In [10]:
poi_listB = list(poi_dataB['category'])
poi_list_strB = []

for i in range(len(poi_listB)):
    index = poi_listB[i]-1
    name = categories[index]
    poi_list_strB.append(name)
    

poi_dataB.drop(columns=['category'], inplace=True)
poi_dataB['category'] = poi_list_strB

In [11]:
poi_listC = list(poi_dataC['category'])
poi_list_strC = []

for i in range(len(poi_listC)):
    index = poi_listC[i]-1
    name = categories[index]
    poi_list_strC.append(name)
    

poi_dataC.drop(columns=['category'], inplace=True)
poi_dataC['category'] = poi_list_strC

In [12]:
poi_listD = list(poi_dataD['category'])
poi_list_strD = []

for i in range(len(poi_listD)):
    index = poi_listD[i]-1
    name = categories[index]
    poi_list_strD.append(name)
    

poi_dataD.drop(columns=['category'], inplace=True)
poi_dataD['category'] = poi_list_strD

In [13]:
def find_nearby_pois(city, predicted_coordinates, radius=1):

    nearby_pois = pd.DataFrame(columns=['x', 'y', 'category'])

    for coordinate in predicted_coordinates:
        predicted_x = coordinate[0]
        predicted_y = coordinate[1]

        if city.upper() == 'A':
            poi_dataA['distance'] = np.sqrt((poi_dataA['x'] - predicted_x)**2 + (poi_dataA['y'] - predicted_y)**2)
            temp_df = poi_dataA[poi_dataA['distance'] <= radius].copy()
            temp_df.drop(columns=['distance', 'POI_count'], inplace=True)

        if city.upper() == 'B':
            poi_dataB['distance'] = np.sqrt((poi_dataB['x'] - predicted_x)**2 + (poi_dataB['y'] - predicted_y)**2)
            temp_df = poi_dataB[poi_dataB['distance'] <= radius].copy()
            temp_df.drop(columns=['distance', 'POI_count'], inplace=True)
        
        if city.upper() == 'C':
            poi_dataC['distance'] = np.sqrt((poi_dataC['x'] - predicted_x)**2 + (poi_dataC['y'] - predicted_y)**2)
            temp_df = poi_dataC[poi_dataC['distance'] <= radius].copy()
            temp_df.drop(columns=['distance', 'POI_count'], inplace=True)

        if city.upper() == 'D':
            poi_dataD['distance'] = np.sqrt((poi_dataD['x'] - predicted_x)**2 + (poi_dataD['y'] - predicted_y)**2)
            temp_df = poi_dataD[poi_dataD['distance'] <= radius].copy()
            temp_df.drop(columns=['distance', 'POI_count'], inplace=True)

        nearby_pois = pd.concat([nearby_pois, temp_df], ignore_index=True)

    return nearby_pois

In [14]:
def find_nearby_pois_A(predicted_coordinates, radius=1):


    predicted_x = predicted_coordinates[0]
    predicted_y = predicted_coordinates[1]

    poi_dataA['distance'] = np.sqrt((poi_dataA['x'] - predicted_x)**2 + (poi_dataA['y'] - predicted_y)**2)
    
    nearby_pois = poi_dataA[poi_dataA['distance'] <= radius].copy()
    nearby_pois.drop(columns=['distance', 'POI_count'], inplace=True)

    return nearby_pois


In [15]:
def find_nearby_pois_B(predicted_coordinates, radius=1):


    predicted_x = predicted_coordinates[0]
    predicted_y = predicted_coordinates[1]

    poi_dataB['distance'] = np.sqrt((poi_dataB['x'] - predicted_x)**2 + (poi_dataB['y'] - predicted_y)**2)
    
    nearby_pois = poi_dataB[poi_dataB['distance'] <= radius].copy()
    nearby_pois.drop(columns=['distance', 'POI_count'], inplace=True)

    return nearby_pois

In [16]:
def find_nearby_pois_C(predicted_coordinates, radius=1):


    predicted_x = predicted_coordinates[0]
    predicted_y = predicted_coordinates[1]

    poi_dataC['distance'] = np.sqrt((poi_dataC['x'] - predicted_x)**2 + (poi_dataC['y'] - predicted_y)**2)
    
    nearby_pois = poi_dataC[poi_dataC['distance'] <= radius].copy()
    nearby_pois.drop(columns=['distance', 'POI_count'], inplace=True)

    return nearby_pois

In [17]:
def find_nearby_pois_D(predicted_coordinates, radius=1):


    predicted_x = predicted_coordinates[0]
    predicted_y = predicted_coordinates[1]

    poi_dataD['distance'] = np.sqrt((poi_dataD['x'] - predicted_x)**2 + (poi_dataD['y'] - predicted_y)**2)
    
    nearby_pois = poi_dataD[poi_dataD['distance'] <= radius].copy()
    nearby_pois.drop(columns=['distance', 'POI_count'], inplace=True)

    return nearby_pois

In [18]:
list_of_predicted_coordinatesA = make_multiple_predictions([[176, 48]], 5, city='A')
find_nearby_pois('A', list_of_predicted_coordinatesA, 1)


,x,y,category
0,175,48,Home Appliances
1,175,48,Fishing
2,175,48,Park
3,175,48,Heavy Industry
4,175,48,Chiropractic
...,...,...,...
270,191,17,Cemetary
271,192,16,Heavy Industry
272,192,16,Real Estate
273,192,16,Building Material


In [19]:
list_of_predicted_coordinatesB= make_multiple_predictions([[79, 73]], 5, city='B')
find_nearby_pois('B', list_of_predicted_coordinatesB, 1)

,x,y,category
0,78,73,Building Material
1,78,73,Real Estate
2,78,73,Heavy Industry
3,78,73,Clothes Store
4,78,73,Driving School
...,...,...,...
432,73,77,Laundry
433,73,77,Convenience Store
434,73,77,Grocery Store
435,73,77,Fishing


In [20]:
list_of_predicted_coordinatesC = make_multiple_predictions([[31, 190]], 5, 'C')
find_nearby_pois('C', list_of_predicted_coordinatesC, 1)

,x,y,category
0,30,190,Cram School
1,30,190,Transit Station
2,30,190,Park
3,30,190,Chiropractic
4,30,190,NPO
...,...,...,...
275,36,161,Driving School
276,36,161,Accountant Office
277,36,161,Home Appliances
278,36,161,Heavy Industry


In [21]:
list_of_predicted_coordinatesD = make_multiple_predictions([[123, 62]], 5, city='D')
find_nearby_pois('D', list_of_predicted_coordinatesD, 1)


,x,y,category
0,122,62,Church
1,122,62,Building Material
2,122,62,Home Appliances
3,122,62,Hair Salon
4,122,62,Heavy Industry
...,...,...,...
149,133,60,School
150,133,62,Sports Recreation
151,134,61,Elderly Care Home
152,134,61,Bar


In [22]:
list_of_suggestionsA = find_nearby_pois_A(make_prediction_city_A([[176, 48]]))
print(list_of_suggestionsA)

          x   y             category
193105  178  42                 Park
193106  178  42    Building Material
193107  178  42          Real Estate
194503  179  41        Interior Shop
194504  179  41                  Vet
194505  179  41               Church
194506  179  41        Swimming Pool
194507  179  41  Japanese restaurant
194508  179  41             Laundry 
194509  179  42       Heavy Industry
194510  179  42                  NPO
194511  179  42              Fishing
194512  179  42     Community Center
194513  179  42                 Port
194514  179  42       Driving School
194515  179  43                  NPO
194516  179  43           Hair Salon
194517  179  43    Elderly Care Home
194518  179  43    Accountant Office
194519  179  43          Real Estate
194520  179  43             Laundry 
194521  179  43                 Bank
194522  179  43       Driving School
194523  179  43                  Vet
194524  179  43                 Café
194525  179  43             Pharmacy
1

In [23]:
list_of_suggestionsB = find_nearby_pois_B(make_prediction_city_B([[79, 73]]))
print(list_of_suggestionsB)

        x   y           category
22847  75  74        Real Estate
22848  75  74  Accountant Office
22849  75  74           Hospital
22850  75  74  Building Material
22851  75  74      Grocery Store
...    ..  ..                ...
25473  77  74  Research Facility
25474  77  74     Heavy Industry
25475  77  74      Clothes Store
25476  77  74  Elderly Care Home
25477  77  74      Lawyer Office

[117 rows x 3 columns]


In [24]:
list_of_suggestionsC = find_nearby_pois_C(make_prediction_city_C([[31, 190]]))
print(list_of_suggestionsC)

        x    y           category
21969  30  185        Real Estate
21970  30  185               Park
21971  30  185               Bank
21972  30  185    Transit Station
21973  30  185                NPO
...    ..  ...                ...
24434  32  185     Heavy Industry
24435  32  185             Bakery
24436  32  185        Cram School
24437  32  185  Convenience Store
24438  32  185               Café

[92 rows x 3 columns]


In [25]:
list_of_suggestionsD = find_nearby_pois_D(make_prediction_city_D([[123, 62]]))
print(list_of_suggestionsD)

         x   y           category
38035  124  61             Church
38036  124  61  Electronics Store
38037  124  61              Pizza
38038  124  61     Utility Copany
38039  124  61      Grocery Store
38040  124  61           Hospital
38372  125  61    Transit Station
38373  125  61        Post Office
38374  125  61  Accountant Office
38375  125  61               Port
38376  125  61         Drug Store
38377  125  61         Hair Salon
38378  125  61           Laundry 
38379  125  61           Hospital
38380  125  61  Building Material
38381  125  62           Laundry 
38382  125  62         Hair Salon
38383  125  62    Home Appliances
38851  126  61    Home Appliances
38852  126  61            Fishing
38853  126  61  Building Material
38854  126  61      Grocery Store
